# Libraries

In [9]:
import tensorflow as tf
from tensorflow import keras
from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from keras.preprocessing import image
import numpy as np

# Functions

In [ ]:
# Load pre-trained ResNet50 model + higher level layers
model = ResNet50(weights='imagenet')

def classify_cat_species(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    
    # Convert image to array and preprocess
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    # Run prediction for top 3 classes
    preds = model.predict(x)
    decoded = decode_predictions(preds, top=3)[0]

    # Filter predictions to only those related to cats
    cat_breeds = [label for label in decoded] # [label for label in decoded if 'cat' in label[1] or 'Egyptian' in label[1] or 'Persian' in label[1]]
    
    print("Top predictions:")
    for pred in cat_breeds:
        print(f"{pred[1]}: {round(pred[2]*100, 2)}%")


102967424/102967424 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 778ms/step
35363/35363 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
Top predictions:
tabby: 57.41%
tiger_cat: 25.08%
Egyptian_cat: 9.0%


# Analyze

In [ ]:
img_path = './postprocessed_data/labeled_imgs/Chausie/Chausie_11.jpg'
classify_cat_species(img_path)